In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
train = pd.read_csv('trainMaster.csv')

# Target

In [3]:
x = train['comment_text']

y1 = train['toxic']
y2 = train['severe_toxic']
y3 = train['obscene']
y4 = train['threat']
y5 = train['insult']
y6 = train['identity_hate']
y7 = train['clean']

datay = [y1, y2, y3, y4, y5, y6, y7]

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
dSplit = {}
for j in range(1,8):
        X_train, X_test, dSplit['y{}_train'.format(j)], dSplit['y{}_test'.format(j)] = train_test_split(x, datay[j-1], test_size=0.3, random_state=101)

In [6]:
print('OwO')

OwO


# K Fold

In [7]:
# from sklearn.model_selection import KFold

In [8]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [9]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer


# Analyzer for Pipeline

In [11]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

# Pipeline

In [12]:
listAnalyzer = [process_normal, process_nocase,process_nopunc,process_nocasepunc]
nama = ['normal','noCase','noPunc','noCasePunc']
dPP = {}
dPP2 = {}

# buat analyzer
for i in range(0,4): 
#     buat n_gram
    for j in range(1,4):
        dPP['pp_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('classifier', GradientBoostingClassifier()),
        ])

        # With TFIDF
        dPP2['pp2_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('tfidf', TfidfTransformer()),
            ('classifier', GradientBoostingClassifier()),
        ])

In [13]:
print('OwO')

OwO


# Predict

In [14]:
ppKey = list(dPP.keys())
pp2Key = list(dPP2.keys())

dPredict_train = {}
dPredict_test = {}
dPredictProba_train = {}
dPredictProba_test = {}

dPredict2_train = {}
dPredict2_test = {}
dPredict2Proba_train = {}
dPredict2Proba_test = {}

for item, item2 in zip(ppKey, pp2Key):
    for i in range(1,8):
#         Pipeline
        dPP[item].fit(X_train, dSplit['y{}_train'.format(i)])
        
        dPredict_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict(X_train)
        dPredictProba_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_train)
        
        dPredict_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict(X_test)
        dPredictProba_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_test)
        
#         Pipeline2
        dPP2[item2].fit(X_train, dSplit['y{}_train'.format(i)])
    
        dPredict2_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_train)
        dPredict2Proba_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_train)
        
        dPredict2_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_test)
        dPredict2Proba_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_test)

In [15]:
print('OwO')

OwO


# Report

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

In [17]:
def calc_train_error(X_train, y_train, predictTrain, predictProbaTrain):
    matt = matthews_corrcoef(y_train, predictTrain)
    f1 = f1_score(y_train, predictTrain, average = 'macro')
    report = classification_report(y_train, predictTrain)
    roc_auc = roc_auc_score(y_train, predictProbaTrain[:,1])
    accuracy = accuracy_score(y_train, predictTrain)
    confMatrix = confusion_matrix(y_train, predictTrain)
    logloss = log_loss(y_train, predictProbaTrain)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_validation_error(X_test, y_test, predictTest, predictProbaTest):
    matt = matthews_corrcoef(y_test, predictTest)
    f1 = f1_score(y_test, predictTest, average = 'macro')
    report = classification_report(y_test, predictTest)
    roc_auc = roc_auc_score(y_test, predictProbaTest[:,1])
    accuracy = accuracy_score(y_test, predictTest)
    confMatrix = confusion_matrix(y_test, predictTest)
    logloss = log_loss(y_test, predictProbaTest)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_metrics(X_train, y_train, X_test, y_test, predictTrain, predictProbaTrain, predictTest, predictProbaTest):
    train_error = calc_train_error(X_train, y_train, predictTrain, predictProbaTrain)
    validation_error = calc_validation_error(X_test, y_test, predictTest, predictProbaTest)
    return train_error, validation_error

## Toxic

In [18]:
dToxic_train = {}
dToxic_test = {}
dToxic2_train = {}
dToxic2_test = {}

for item in ppKey:
    dToxic_train['hasil_train_{}'.format(item)], dToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y1_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y1_{}'.format(item)])

for item2 in pp2Key:
    dToxic2_train['hasil_train_{}'.format(item2)], dToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict2_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y1_{}'.format(item2)])

In [19]:
dToxic_train_key = list(dToxic_train.keys())
dToxic_test_key = list(dToxic_test.keys())
listItem = []

for train, test in zip(dToxic_train_key, dToxic_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dToxic_train[train]['accuracy'],
                     dToxic_test[test]['accuracy'],
                     dToxic_train[train]['roc'],
                     dToxic_test[test]['roc'],
                     dToxic_train[train]['f1'],
                     dToxic_test[test]['f1'],
                     dToxic_train[train]['matthew'],
                     dToxic_test[test]['matthew'],
                     dToxic_train[train]['logloss'],
                     dToxic_test[test]['logloss'],])


dfToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.930939,0.930419,0.858691,0.855804,0.704425,0.706820,0.507056,0.505925,0.215883,0.218185
1,gbc_hasil_test_pp_normal_ngram_2,0.930939,0.930419,0.858690,0.855805,0.704425,0.706820,0.507056,0.505925,0.215884,0.218179
2,gbc_hasil_test_pp_normal_ngram_3,0.930939,0.930419,0.858690,0.855801,0.704425,0.706820,0.507056,0.505925,0.215883,0.218194
3,gbc_hasil_test_pp_noCase_ngram_1,0.933759,0.932988,0.877810,0.874723,0.723835,0.725028,0.533994,0.530505,0.203648,0.205399
4,gbc_hasil_test_pp_noCase_ngram_2,0.933786,0.933218,0.876861,0.873335,0.724108,0.725930,0.534236,0.532652,0.203820,0.205325
5,gbc_hasil_test_pp_noCase_ngram_3,0.933751,0.933260,0.875520,0.871599,0.723887,0.726101,0.533900,0.533043,0.203609,0.205330
6,gbc_hasil_test_pp_noPunc_ngram_1,0.933187,0.932821,0.884479,0.882348,0.720463,0.722771,0.528568,0.528938,0.202603,0.204088
7,gbc_hasil_test_pp_noPunc_ngram_2,0.934118,0.933761,0.882831,0.879790,0.727314,0.729289,0.537257,0.537708,0.202599,0.204258
8,gbc_hasil_test_pp_noPunc_ngram_3,0.934118,0.933782,0.882827,0.879800,0.727314,0.729414,0.537257,0.537902,0.202602,0.204257
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.937054,0.937166,0.900922,0.896442,0.746678,0.750624,0.564067,0.568535,0.187688,0.188872


In [20]:
dToxic2_train_key = list(dToxic2_train.keys())
dToxic2_test_key = list(dToxic2_test.keys())
listItem = []

for train, test in zip(dToxic2_train_key, dToxic2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dToxic2_train[train]['accuracy'],
                     dToxic2_test[test]['accuracy'],
                     dToxic2_train[train]['roc'],
                     dToxic2_test[test]['roc'],
                     dToxic2_train[train]['f1'],
                     dToxic2_test[test]['f1'],
                     dToxic2_train[train]['matthew'],
                     dToxic2_test[test]['matthew'],
                     dToxic2_train[train]['logloss'],
                     dToxic2_test[test]['logloss'],])


dfToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.932569,0.932445,0.857336,0.855392,0.713435,0.718684,0.523115,0.525477,0.213718,0.215774
1,gbc_hasil_test_pp2_normal_ngram_2,0.932551,0.932403,0.858248,0.855070,0.713321,0.718339,0.522944,0.525088,0.213630,0.215802
2,gbc_hasil_test_pp2_normal_ngram_3,0.932569,0.932445,0.857336,0.855377,0.713435,0.718684,0.523115,0.525477,0.213718,0.215788
3,gbc_hasil_test_pp2_noCase_ngram_1,0.936356,0.936309,0.877155,0.875397,0.737931,0.743655,0.558151,0.560921,0.200117,0.201765
4,gbc_hasil_test_pp2_noCase_ngram_2,0.936418,0.936288,0.876196,0.874693,0.738340,0.743533,0.558713,0.560734,0.200175,0.201724
5,gbc_hasil_test_pp2_noCase_ngram_3,0.935998,0.935808,0.878113,0.877286,0.735642,0.740806,0.554926,0.556420,0.200017,0.201551
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.936669,0.936288,0.884026,0.878866,0.739872,0.742858,0.560969,0.560764,0.198955,0.200848
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.936821,0.936435,0.882918,0.878862,0.740878,0.743935,0.562323,0.562059,0.199101,0.200849
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.936714,0.936309,0.882403,0.878522,0.740105,0.742830,0.561378,0.560960,0.199177,0.200812
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.941700,0.942054,0.902824,0.901094,0.770960,0.777715,0.604545,0.610427,0.182602,0.183795


In [21]:
# print(confusion_matrix(dSplit['y1_test'], predictions))
# print('\n')
# print(classification_report(dSplit['y1_test'], predictions))

## Severe Toxic

In [22]:
dSToxic_train = {}
dSToxic_test = {}
dSToxic2_train = {}
dSToxic2_test = {}

for item in ppKey:
    dSToxic_train['hasil_train_{}'.format(item)], dSToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y2_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y2_{}'.format(item)])

for item2 in pp2Key:
    dSToxic2_train['hasil_train_{}'.format(item2)], dSToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict2_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y2_{}'.format(item2)])

In [23]:
dSToxic_train_key = list(dSToxic_train.keys())
dSToxic_test_key = list(dSToxic_test.keys())
listItem = []

for train, test in zip(dSToxic_train_key, dSToxic_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dSToxic_train[train]['accuracy'],
                     dSToxic_test[test]['accuracy'],
                     dSToxic_train[train]['roc'],
                     dSToxic_test[test]['roc'],
                     dSToxic_train[train]['f1'],
                     dSToxic_test[test]['f1'],
                     dSToxic_train[train]['matthew'],
                     dSToxic_test[test]['matthew'],
                     dSToxic_train[train]['logloss'],
                     dSToxic_test[test]['logloss'],])


dfSToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.992811,0.989347,0.952191,0.902848,0.730345,0.600122,0.519520,0.245731,0.025242,0.042359
1,gbc_hasil_test_pp_normal_ngram_2,0.992623,0.989159,0.950992,0.903766,0.719011,0.594946,0.500947,0.231709,0.025142,0.045271
2,gbc_hasil_test_pp_normal_ngram_3,0.992757,0.989472,0.946579,0.903033,0.724311,0.601124,0.512703,0.251442,0.025731,0.040518
3,gbc_hasil_test_pp_noCase_ngram_1,0.992677,0.989702,0.971277,0.915803,0.722179,0.605517,0.506257,0.266604,0.023214,0.041566
4,gbc_hasil_test_pp_noCase_ngram_2,0.992551,0.989723,0.970604,0.923173,0.717802,0.609401,0.495706,0.273039,0.023412,0.042821
5,gbc_hasil_test_pp_noCase_ngram_3,0.992677,0.989743,0.971277,0.916824,0.722179,0.607117,0.506257,0.270603,0.023214,0.041275
6,gbc_hasil_test_pp_noPunc_ngram_1,0.992551,0.989535,0.975908,0.930482,0.724080,0.627350,0.500191,0.293353,0.024117,0.042781
7,gbc_hasil_test_pp_noPunc_ngram_2,0.992516,0.989535,0.973837,0.928807,0.717955,0.632736,0.493669,0.301506,0.024695,0.043052
8,gbc_hasil_test_pp_noPunc_ngram_3,0.992489,0.989451,0.975960,0.932476,0.719001,0.628731,0.492892,0.293235,0.024065,0.044199
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.992659,0.989514,0.974056,0.947217,0.724454,0.615901,0.506564,0.275409,0.021904,0.039161


In [24]:
dSToxic2_train_key = list(dSToxic2_train.keys())
dSToxic2_test_key = list(dSToxic2_test.keys())
listItem = []

for train, test in zip(dSToxic2_train_key, dSToxic2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dSToxic2_train[train]['accuracy'],
                     dSToxic2_test[test]['accuracy'],
                     dSToxic2_train[train]['roc'],
                     dSToxic2_test[test]['roc'],
                     dSToxic2_train[train]['f1'],
                     dSToxic2_test[test]['f1'],
                     dSToxic2_train[train]['matthew'],
                     dSToxic2_test[test]['matthew'],
                     dSToxic2_train[train]['logloss'],
                     dSToxic2_test[test]['logloss'],])


dfSToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.992972,0.989284,0.936663,0.908769,0.736390,0.619540,0.533112,0.274692,0.024949,0.044252
1,gbc_hasil_test_pp2_normal_ngram_2,0.992910,0.989179,0.936190,0.910823,0.732800,0.613003,0.527181,0.261675,0.025159,0.042057
2,gbc_hasil_test_pp2_normal_ngram_3,0.992972,0.989263,0.936663,0.908738,0.736390,0.619355,0.533112,0.273882,0.024949,0.044236
3,gbc_hasil_test_pp2_noCase_ngram_1,0.992865,0.989743,0.966508,0.932107,0.740275,0.652128,0.529119,0.335816,0.022364,0.040631
4,gbc_hasil_test_pp2_noCase_ngram_2,0.992936,0.989639,0.967534,0.934121,0.745499,0.648095,0.536719,0.327288,0.022106,0.040569
5,gbc_hasil_test_pp2_noCase_ngram_3,0.992936,0.989660,0.967534,0.935109,0.745499,0.648312,0.536719,0.328111,0.022106,0.040640
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.992963,0.989430,0.958660,0.924917,0.746950,0.644970,0.539185,0.317847,0.023933,0.039952
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.992990,0.989639,0.960597,0.927583,0.745165,0.649076,0.539361,0.328767,0.023608,0.038371
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.992918,0.989367,0.961191,0.932242,0.745501,0.640379,0.535801,0.309355,0.023404,0.042183
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.993294,0.989075,0.990943,0.947850,0.766710,0.635561,0.570543,0.295898,0.020449,0.052177


## Obscene

In [25]:
dObscene_train = {}
dObscene_test = {}
dObscene2_train = {}
dObscene2_test = {}

for item in ppKey:
    dObscene_train['hasil_train_{}'.format(item)], dObscene_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y3_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y3_{}'.format(item)])

for item2 in pp2Key:
    dObscene2_train['hasil_train_{}'.format(item2)], dObscene2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict2_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y3_{}'.format(item2)])

In [26]:
dObscene_train_key = list(dObscene_train.keys())
dObscene_test_key = list(dObscene_test.keys())
listItem = []

for train, test in zip(dObscene_train_key, dObscene_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfObsceneResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObsceneResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.969489,0.968332,0.901078,0.894975,0.796291,0.794868,0.637947,0.631532,0.108694,0.113628
1,gbc_hasil_test_pp_normal_ngram_2,0.969489,0.968353,0.901078,0.894983,0.796291,0.794953,0.637947,0.631804,0.108695,0.113589
2,gbc_hasil_test_pp_normal_ngram_3,0.969489,0.968353,0.901078,0.894982,0.796291,0.794953,0.637947,0.631804,0.108694,0.113604
3,gbc_hasil_test_pp_noCase_ngram_1,0.971378,0.970672,0.926286,0.919051,0.816010,0.817459,0.665993,0.665572,0.098108,0.103564
4,gbc_hasil_test_pp_noCase_ngram_2,0.971378,0.970651,0.926286,0.919113,0.816010,0.817373,0.665993,0.665320,0.098108,0.103554
5,gbc_hasil_test_pp_noCase_ngram_3,0.971540,0.970463,0.924157,0.917294,0.817239,0.815984,0.668224,0.662786,0.097750,0.103459
6,gbc_hasil_test_pp_noPunc_ngram_1,0.972130,0.970630,0.920377,0.916166,0.821824,0.817112,0.676388,0.664994,0.095979,0.100988
7,gbc_hasil_test_pp_noPunc_ngram_2,0.972130,0.970630,0.920377,0.916165,0.821824,0.817112,0.676388,0.664994,0.095979,0.100988
8,gbc_hasil_test_pp_noPunc_ngram_3,0.972130,0.970630,0.920377,0.916169,0.821824,0.817112,0.676388,0.664994,0.095979,0.101002
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.973903,0.972656,0.954266,0.950040,0.838514,0.834954,0.701479,0.693198,0.082819,0.087973


In [27]:
dObscene2_train_key = list(dObscene2_train.keys())
dObscene2_test_key = list(dObscene2_test.keys())
listItem = []

for train, test in zip(dObscene2_train_key, dObscene2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfObscene2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObscene2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.970125,0.969502,0.899900,0.892970,0.799851,0.803424,0.646931,0.647845,0.106568,0.111818
1,gbc_hasil_test_pp2_normal_ngram_2,0.970125,0.969481,0.899857,0.893463,0.799895,0.803532,0.646935,0.647619,0.106624,0.111676
2,gbc_hasil_test_pp2_normal_ngram_3,0.970125,0.969523,0.899858,0.893468,0.799895,0.803704,0.646935,0.648165,0.106624,0.111643
3,gbc_hasil_test_pp2_noCase_ngram_1,0.973115,0.971800,0.924157,0.914832,0.828084,0.825972,0.689540,0.680732,0.094526,0.100849
4,gbc_hasil_test_pp2_noCase_ngram_2,0.972945,0.971507,0.924952,0.917466,0.826540,0.823333,0.687175,0.676656,0.094664,0.100750
5,gbc_hasil_test_pp2_noCase_ngram_3,0.972703,0.971570,0.923806,0.914233,0.824453,0.823680,0.683825,0.677449,0.094736,0.100993
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.973706,0.972009,0.919659,0.917379,0.833090,0.826770,0.697642,0.683237,0.092940,0.098137
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.973652,0.972134,0.919974,0.916388,0.832505,0.827791,0.696883,0.684933,0.092640,0.097748
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.973706,0.972009,0.919659,0.917388,0.833090,0.826770,0.697642,0.683237,0.092940,0.098031
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.977144,0.975602,0.949940,0.944603,0.862685,0.858552,0.743812,0.732269,0.078619,0.083825


## Threat

In [28]:
dThreat_train = {}
dThreat_test = {}
dThreat2_train = {}
dThreat2_test = {}

for item in ppKey:
    dThreat_train['hasil_train_{}'.format(item)], dThreat_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y4_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y4_{}'.format(item)])

for item2 in pp2Key:
    dThreat2_train['hasil_train_{}'.format(item2)], dThreat2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict2_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y4_{}'.format(item2)])

In [29]:
dThreat_train_key = list(dThreat_train.keys())
dThreat_test_key = list(dThreat_test.keys())
listItem = []

for train, test in zip(dThreat_train_key, dThreat_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfThreatResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreatResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.969489,0.968332,0.901078,0.894975,0.796291,0.794868,0.637947,0.631532,0.108694,0.113628
1,gbc_hasil_test_pp_normal_ngram_2,0.969489,0.968353,0.901078,0.894983,0.796291,0.794953,0.637947,0.631804,0.108695,0.113589
2,gbc_hasil_test_pp_normal_ngram_3,0.969489,0.968353,0.901078,0.894982,0.796291,0.794953,0.637947,0.631804,0.108694,0.113604
3,gbc_hasil_test_pp_noCase_ngram_1,0.971378,0.970672,0.926286,0.919051,0.816010,0.817459,0.665993,0.665572,0.098108,0.103564
4,gbc_hasil_test_pp_noCase_ngram_2,0.971378,0.970651,0.926286,0.919113,0.816010,0.817373,0.665993,0.665320,0.098108,0.103554
5,gbc_hasil_test_pp_noCase_ngram_3,0.971540,0.970463,0.924157,0.917294,0.817239,0.815984,0.668224,0.662786,0.097750,0.103459
6,gbc_hasil_test_pp_noPunc_ngram_1,0.972130,0.970630,0.920377,0.916166,0.821824,0.817112,0.676388,0.664994,0.095979,0.100988
7,gbc_hasil_test_pp_noPunc_ngram_2,0.972130,0.970630,0.920377,0.916165,0.821824,0.817112,0.676388,0.664994,0.095979,0.100988
8,gbc_hasil_test_pp_noPunc_ngram_3,0.972130,0.970630,0.920377,0.916169,0.821824,0.817112,0.676388,0.664994,0.095979,0.101002
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.973903,0.972656,0.954266,0.950040,0.838514,0.834954,0.701479,0.693198,0.082819,0.087973


In [30]:
dThreat2_train_key = list(dThreat2_train.keys())
dThreat2_test_key = list(dThreat2_test.keys())
listItem = []

for train, test in zip(dThreat2_train_key, dThreat2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfThreat2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreat2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.970125,0.969502,0.899900,0.892970,0.799851,0.803424,0.646931,0.647845,0.106568,0.111818
1,gbc_hasil_test_pp2_normal_ngram_2,0.970125,0.969481,0.899857,0.893463,0.799895,0.803532,0.646935,0.647619,0.106624,0.111676
2,gbc_hasil_test_pp2_normal_ngram_3,0.970125,0.969523,0.899858,0.893468,0.799895,0.803704,0.646935,0.648165,0.106624,0.111643
3,gbc_hasil_test_pp2_noCase_ngram_1,0.973115,0.971800,0.924157,0.914832,0.828084,0.825972,0.689540,0.680732,0.094526,0.100849
4,gbc_hasil_test_pp2_noCase_ngram_2,0.972945,0.971507,0.924952,0.917466,0.826540,0.823333,0.687175,0.676656,0.094664,0.100750
5,gbc_hasil_test_pp2_noCase_ngram_3,0.972703,0.971570,0.923806,0.914233,0.824453,0.823680,0.683825,0.677449,0.094736,0.100993
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.973706,0.972009,0.919659,0.917379,0.833090,0.826770,0.697642,0.683237,0.092940,0.098137
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.973652,0.972134,0.919974,0.916388,0.832505,0.827791,0.696883,0.684933,0.092640,0.097748
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.973706,0.972009,0.919659,0.917388,0.833090,0.826770,0.697642,0.683237,0.092940,0.098031
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.977144,0.975602,0.949940,0.944603,0.862685,0.858552,0.743812,0.732269,0.078619,0.083825


## Insult

In [31]:
dInsult_train = {}
dInsult_test = {}
dInsult2_train = {}
dInsult2_test = {}

for item in ppKey:
    dInsult_train['hasil_train_{}'.format(item)], dInsult_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y5_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y5_{}'.format(item)])

for item2 in pp2Key:
    dInsult2_train['hasil_train_{}'.format(item2)], dInsult2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict2_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y5_{}'.format(item2)])

In [32]:
dInsult_train_key = list(dInsult_train.keys())
dInsult_test_key = list(dInsult_test.keys())
listItem = []

for train, test in zip(dInsult_train_key, dInsult_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dInsult_train[train]['accuracy'],
                     dInsult_test[test]['accuracy'],
                     dInsult_train[train]['roc'],
                     dInsult_test[test]['roc'],
                     dInsult_train[train]['f1'],
                     dInsult_test[test]['f1'],
                     dInsult_train[train]['matthew'],
                     dInsult_test[test]['matthew'],
                     dInsult_train[train]['logloss'],
                     dInsult_test[test]['logloss'],])


dfInsultResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsultResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.962829,0.962045,0.890347,0.884560,0.710075,0.709171,0.490491,0.481502,0.121677,0.126011
1,gbc_hasil_test_pp_normal_ngram_2,0.962829,0.962191,0.889931,0.884024,0.709848,0.710723,0.490387,0.484290,0.121670,0.125910
2,gbc_hasil_test_pp_normal_ngram_3,0.962829,0.962191,0.889931,0.884020,0.709848,0.710723,0.490387,0.484290,0.121670,0.125934
3,gbc_hasil_test_pp_noCase_ngram_1,0.965049,0.964301,0.908526,0.901222,0.752066,0.752506,0.542641,0.537605,0.114005,0.118019
4,gbc_hasil_test_pp_noCase_ngram_2,0.965058,0.964572,0.908907,0.901813,0.752275,0.753928,0.542891,0.541170,0.113869,0.117938
5,gbc_hasil_test_pp_noCase_ngram_3,0.965049,0.964280,0.908525,0.901203,0.752066,0.752295,0.542641,0.537238,0.114009,0.118068
6,gbc_hasil_test_pp_noPunc_ngram_1,0.963957,0.963402,0.910134,0.907885,0.726082,0.726349,0.513634,0.508657,0.113423,0.116358
7,gbc_hasil_test_pp_noPunc_ngram_2,0.963948,0.963507,0.910878,0.908735,0.726118,0.728157,0.513534,0.511011,0.113020,0.116196
8,gbc_hasil_test_pp_noPunc_ngram_3,0.964019,0.963507,0.909056,0.906397,0.727077,0.728314,0.514958,0.511109,0.113274,0.116345
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.965944,0.965199,0.935004,0.928906,0.753894,0.753647,0.553189,0.546677,0.103138,0.106171


In [33]:
dInsult2_train_key = list(dInsult2_train.keys())
dInsult2_test_key = list(dInsult2_test.keys())
listItem = []

for train, test in zip(dInsult2_train_key, dInsult2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dInsult2_train[train]['accuracy'],
                     dInsult2_test[test]['accuracy'],
                     dInsult2_train[train]['roc'],
                     dInsult2_test[test]['roc'],
                     dInsult2_train[train]['f1'],
                     dInsult2_test[test]['f1'],
                     dInsult2_train[train]['matthew'],
                     dInsult2_test[test]['matthew'],
                     dInsult2_train[train]['logloss'],
                     dInsult2_test[test]['logloss'],])


dfInsult2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsult2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.963482,0.962650,0.888312,0.883271,0.714912,0.714410,0.502067,0.492323,0.119489,0.124212
1,gbc_hasil_test_pp2_normal_ngram_2,0.963545,0.963068,0.890010,0.884913,0.716622,0.718610,0.503658,0.500056,0.119415,0.124381
2,gbc_hasil_test_pp2_normal_ngram_3,0.963563,0.962859,0.888309,0.884766,0.715875,0.716345,0.503633,0.496108,0.119653,0.123822
3,gbc_hasil_test_pp2_noCase_ngram_1,0.966141,0.965345,0.912299,0.906117,0.758906,0.759618,0.558544,0.552791,0.110463,0.115480
4,gbc_hasil_test_pp2_noCase_ngram_2,0.966087,0.965199,0.912714,0.905493,0.758295,0.757760,0.557571,0.549936,0.110617,0.115840
5,gbc_hasil_test_pp2_noCase_ngram_3,0.966096,0.965282,0.912426,0.906026,0.758444,0.759248,0.557765,0.551927,0.110489,0.115549
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.965479,0.964405,0.902707,0.900911,0.740737,0.739002,0.540677,0.528200,0.110289,0.113951
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.965479,0.964426,0.902707,0.900577,0.740737,0.739230,0.540677,0.528581,0.110289,0.113957
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.965479,0.964426,0.902707,0.900913,0.740737,0.739230,0.540677,0.528581,0.110289,0.113934
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.968361,0.967476,0.924396,0.923170,0.774342,0.774156,0.592161,0.583850,0.099639,0.103476


## Identity Hate

In [34]:
dIdHate_train = {}
dIdHate_test = {}
dIdHate2_train = {}
dIdHate2_test = {}

for item in ppKey:
    dIdHate_train['hasil_train_{}'.format(item)], dIdHate_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y6_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y6_{}'.format(item)])

for item2 in pp2Key:
    dIdHate2_train['hasil_train_{}'.format(item2)], dIdHate2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict2_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y6_{}'.format(item2)])

In [35]:
dIdHate_train_key = list(dIdHate_train.keys())
dIdHate_test_key = list(dIdHate_test.keys())
listItem = []

for train, test in zip(dIdHate_train_key, dIdHate_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dIdHate_train[train]['accuracy'],
                     dIdHate_test[test]['accuracy'],
                     dIdHate_train[train]['roc'],
                     dIdHate_test[test]['roc'],
                     dIdHate_train[train]['f1'],
                     dIdHate_test[test]['f1'],
                     dIdHate_train[train]['matthew'],
                     dIdHate_test[test]['matthew'],
                     dIdHate_train[train]['logloss'],
                     dIdHate_test[test]['logloss'],])


dfIdHateResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHateResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.994342,0.991791,0.891337,0.811889,0.776041,0.634720,0.603904,0.316251,0.024840,0.040562
1,gbc_hasil_test_pp_normal_ngram_2,0.994342,0.991853,0.895665,0.815399,0.777597,0.640809,0.604462,0.327254,0.024647,0.040344
2,gbc_hasil_test_pp_normal_ngram_3,0.994315,0.991644,0.892829,0.815428,0.774821,0.630252,0.601528,0.304009,0.024765,0.041553
3,gbc_hasil_test_pp_noCase_ngram_1,0.994369,0.991582,0.935903,0.856980,0.782728,0.648663,0.608516,0.329001,0.022551,0.041511
4,gbc_hasil_test_pp_noCase_ngram_2,0.994423,0.991686,0.927156,0.851137,0.784801,0.650007,0.612970,0.334189,0.022375,0.042555
5,gbc_hasil_test_pp_noCase_ngram_3,0.994485,0.991686,0.932555,0.855859,0.786783,0.645072,0.617980,0.326992,0.022676,0.042704
6,gbc_hasil_test_pp_noPunc_ngram_1,0.994315,0.991644,0.935322,0.856964,0.781846,0.660060,0.604704,0.347880,0.022592,0.044266
7,gbc_hasil_test_pp_noPunc_ngram_2,0.994279,0.991644,0.932732,0.858344,0.777453,0.656601,0.600158,0.342693,0.022997,0.043878
8,gbc_hasil_test_pp_noPunc_ngram_3,0.994315,0.991331,0.934399,0.857034,0.780356,0.644327,0.603907,0.315550,0.022867,0.046419
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.994521,0.991916,0.955998,0.888140,0.795164,0.677071,0.624542,0.380452,0.020467,0.042106


In [36]:
dIdHate2_train_key = list(dIdHate2_train.keys())
dIdHate2_test_key = list(dIdHate2_test.keys())
listItem = []

for train, test in zip(dIdHate2_train_key, dIdHate2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dIdHate2_train[train]['accuracy'],
                     dIdHate2_test[test]['accuracy'],
                     dIdHate2_train[train]['roc'],
                     dIdHate2_test[test]['roc'],
                     dIdHate2_train[train]['f1'],
                     dIdHate2_test[test]['f1'],
                     dIdHate2_train[train]['matthew'],
                     dIdHate2_test[test]['matthew'],
                     dIdHate2_train[train]['logloss'],
                     dIdHate2_test[test]['logloss'],])


dfIdHate2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHate2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.994628,0.990913,0.895656,0.827714,0.792887,0.651926,0.630093,0.319552,0.023525,0.053407
1,gbc_hasil_test_pp2_normal_ngram_2,0.994736,0.990934,0.898403,0.826187,0.798947,0.648844,0.639630,0.314723,0.023475,0.050756
2,gbc_hasil_test_pp2_normal_ngram_3,0.994709,0.991059,0.896856,0.823035,0.797785,0.651472,0.637386,0.321286,0.023346,0.051537
3,gbc_hasil_test_pp2_noCase_ngram_1,0.995112,0.991749,0.938442,0.869081,0.818690,0.686900,0.671728,0.391344,0.019906,0.052753
4,gbc_hasil_test_pp2_noCase_ngram_2,0.995094,0.991415,0.937273,0.849927,0.818264,0.674217,0.670424,0.364828,0.019662,0.055136
5,gbc_hasil_test_pp2_noCase_ngram_3,0.995112,0.991644,0.937971,0.866783,0.818214,0.690205,0.671552,0.394612,0.019902,0.055994
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.994995,0.991435,0.927908,0.868256,0.812531,0.667200,0.661892,0.354003,0.020862,0.047614
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.994816,0.991352,0.927129,0.867912,0.805050,0.668206,0.647354,0.353979,0.021584,0.049839
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.994906,0.991686,0.925157,0.859430,0.807650,0.670606,0.654205,0.364889,0.021275,0.043994
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.995121,0.991686,0.969329,0.901690,0.821709,0.677972,0.673586,0.376176,0.017907,0.047758


## Clean

In [37]:
dClean_train = {}
dClean_test = {}
dClean2_train = {}
dClean2_test = {}

for item in ppKey:
    dClean_train['hasil_train_{}'.format(item)], dClean_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y7_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y7_{}'.format(item)])

for item2 in pp2Key:
    dClean2_train['hasil_train_{}'.format(item2)], dClean2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict2_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y7_{}'.format(item2)])

In [38]:
dClean_train_key = list(dClean_train.keys())
dClean_test_key = list(dClean_test.keys())
listItem = []

for train, test in zip(dClean_train_key, dClean_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dClean_train[train]['accuracy'],
                     dClean_test[test]['accuracy'],
                     dClean_train[train]['roc'],
                     dClean_test[test]['roc'],
                     dClean_train[train]['f1'],
                     dClean_test[test]['f1'],
                     dClean_train[train]['matthew'],
                     dClean_test[test]['matthew'],
                     dClean_train[train]['logloss'],
                     dClean_test[test]['logloss'],])


dfCleanResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfCleanResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp_normal_ngram_1,0.927260,0.927097,0.855416,0.850227,0.705532,0.708489,0.511148,0.512234,0.225668,0.227803
1,gbc_hasil_test_pp_normal_ngram_2,0.927188,0.927014,0.856492,0.851601,0.705168,0.708155,0.510475,0.511450,0.225724,0.227692
2,gbc_hasil_test_pp_normal_ngram_3,0.927260,0.927076,0.855416,0.850227,0.705532,0.708363,0.511148,0.512044,0.225668,0.227810
3,gbc_hasil_test_pp_noCase_ngram_1,0.931011,0.930523,0.874859,0.869607,0.728866,0.730208,0.544445,0.542506,0.212760,0.214313
4,gbc_hasil_test_pp_noCase_ngram_2,0.931011,0.930544,0.874859,0.869568,0.728866,0.730326,0.544445,0.542687,0.212760,0.214352
5,gbc_hasil_test_pp_noCase_ngram_3,0.930966,0.930502,0.874447,0.869340,0.728410,0.730240,0.544091,0.542312,0.212806,0.214297
6,gbc_hasil_test_pp_noPunc_ngram_1,0.930644,0.930941,0.879049,0.873679,0.727440,0.731905,0.541137,0.546186,0.210134,0.211598
7,gbc_hasil_test_pp_noPunc_ngram_2,0.930653,0.930941,0.879036,0.873396,0.727459,0.731905,0.541220,0.546186,0.210138,0.211684
8,gbc_hasil_test_pp_noPunc_ngram_3,0.932193,0.932424,0.881024,0.875231,0.737211,0.740953,0.554401,0.558776,0.209848,0.211388
9,gbc_hasil_test_pp_noCasePunc_ngram_1,0.938272,0.938753,0.900051,0.895550,0.771940,0.775938,0.604311,0.610194,0.194067,0.194950


In [39]:
dClean2_train_key = list(dClean2_train.keys())
dClean2_test_key = list(dClean2_test.keys())
listItem = []

for train, test in zip(dClean2_train_key, dClean2_test_key):
    listItem.append(['gbc_{}'.format(test),
                     dClean2_train[train]['accuracy'],
                     dClean2_test[test]['accuracy'],
                     dClean2_train[train]['roc'],
                     dClean2_test[test]['roc'],
                     dClean2_train[train]['f1'],
                     dClean2_test[test]['f1'],
                     dClean2_train[train]['matthew'],
                     dClean2_test[test]['matthew'],
                     dClean2_train[train]['logloss'],
                     dClean2_test[test]['logloss'],])


dfClean2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfClean2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,gbc_hasil_test_pp2_normal_ngram_1,0.929158,0.928894,0.857525,0.851944,0.716388,0.719562,0.528451,0.528342,0.223664,0.225885
1,gbc_hasil_test_pp2_normal_ngram_2,0.929122,0.928810,0.856954,0.852326,0.715998,0.718752,0.528172,0.527642,0.223818,0.225879
2,gbc_hasil_test_pp2_normal_ngram_3,0.928388,0.928100,0.857389,0.852521,0.711586,0.715297,0.521587,0.521209,0.223624,0.225781
3,gbc_hasil_test_pp2_noCase_ngram_1,0.932605,0.932528,0.876109,0.872234,0.737045,0.740675,0.558322,0.559751,0.209579,0.211274
4,gbc_hasil_test_pp2_noCase_ngram_2,0.933169,0.933218,0.875157,0.871973,0.740729,0.744769,0.563000,0.565513,0.209680,0.211136
5,gbc_hasil_test_pp2_noCase_ngram_3,0.933169,0.933239,0.875157,0.871974,0.740729,0.744814,0.563000,0.565696,0.209680,0.211128
6,gbc_hasil_test_pp2_noPunc_ngram_1,0.933840,0.934012,0.881757,0.877184,0.744206,0.748630,0.568699,0.572185,0.207370,0.208306
7,gbc_hasil_test_pp2_noPunc_ngram_2,0.933840,0.933970,0.881757,0.877296,0.744206,0.748402,0.568699,0.571839,0.207370,0.208314
8,gbc_hasil_test_pp2_noPunc_ngram_3,0.933679,0.933615,0.881548,0.877340,0.743062,0.746390,0.567400,0.568897,0.207347,0.208297
9,gbc_hasil_test_pp2_noCasePunc_ngram_1,0.939471,0.940090,0.901836,0.898122,0.775866,0.782133,0.614018,0.620650,0.189989,0.190770


In [40]:
import pickle

In [43]:
# dfToxicResult.to_csv('gbc_toxic.csv',index=False)
# dfToxic2Result.to_csv('gbc_toxic2.csv',index=False)

# dfSToxicResult.to_csv('gbc_stoxic.csv',index=False)
# dfSToxic2Result.to_csv('gbc_stoxic2.csv',index=False)

# dfObsceneResult.to_csv('gbc_obscene.csv',index=False)
# dfObscene2Result.to_csv('gbc_obscene2.csv',index=False)

# dfThreatResult.to_csv('gbc_threat.csv',index=False)
# dfThreat2Result.to_csv('gbc_threat2.csv',index=False)

# dfInsultResult.to_csv('gbc_insult.csv',index=False)
# dfInsult2Result.to_csv('gbc_insult2.csv',index=False)

# dfIdHateResult.to_csv('gbc_idHate.csv',index=False)
# dfIdHate2Result.to_csv('gbc_idHate2.csv',index=False)

# dfCleanResult.to_csv('gbc_clean.csv',index=False)
# dfClean2Result.to_csv('gbc_clean2.csv',index=False)

In [42]:
model = '_gbc'

pickle.dump(dToxic_train, open('dToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic_test, open('dToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_train, open('dToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_test, open('dToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dSToxic_train, open('dSToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic_test, open('dSToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_train, open('dSToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_test, open('dSToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dObscene_train, open('dObscene_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene_test, open('dObscene_test{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_train, open('dObscene2_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_test, open('dObscene2_test{}.sav'.format(model), 'wb'))

pickle.dump(dThreat_train, open('dThreat_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat_test, open('dThreat_test{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_train, open('dThreat2_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_test, open('dThreat2_test{}.sav'.format(model), 'wb'))

pickle.dump(dInsult_train, open('dInsult_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult_test, open('dInsult_test{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_train, open('dInsult2_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_test, open('dInsult2_test{}.sav'.format(model), 'wb'))

pickle.dump(dIdHate_train, open('dIdHate_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate_test, open('dIdHate_test{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_train, open('dIdHate2_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_test, open('dIdHate2_test{}.sav'.format(model), 'wb'))

pickle.dump(dClean_train, open('dClean_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean_test, open('dClean_test{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_train, open('dClean2_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_test, open('dClean2_test{}.sav'.format(model), 'wb'))

In [44]:
pickle.dump(dPredict_train, open('dPredict_train.sav', 'wb'))
pickle.dump(dPredict_test, open('dPredict_test.sav', 'wb'))
pickle.dump(dPredictProba_train, open('dPredictProba_train.sav', 'wb'))
pickle.dump(dPredictProba_test, open('dPredictProba_test.sav', 'wb'))

pickle.dump(dPredict2_train, open('dPredict2_train.sav', 'wb'))
pickle.dump(dPredict2_test, open('dPredict2_test.sav', 'wb'))
pickle.dump(dPredict2Proba_train, open('dPredict2Proba_train.sav', 'wb'))
pickle.dump(dPredict2Proba_test, open('dPredict2Proba_test.sav', 'wb'))

In [45]:
print(1)

1
